<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_t81_559_class7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative AI
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 7 Assignment: LLM Tools**

**Student Name: Your Name**

# Assignment Instructions

A [file](https://data.heatonresearch.com/data/t81-559/assignments/559_numbers_2.csv) is provided that contains a number of equations. You can find this file here.

* https://data.heatonresearch.com/data/t81-559/assignments/559_numbers_2.csv

Sample lines from this file include:

|equation|
|---|
|41748459 - 87226336|
|92995162 * 46769739|
|61530438 * 56074589|
|95329602 + 45418854|
|412907 + 3731910|
|...|

Use an LangChain agent, with a LangChain tool to calculate each of these equations and submit a file similar to this:

|equation|value|
|---|---|
|41748459 - 87226336|-45477877|
|92995162 * 46769739|4349359455002718|
|61530438 * 56074589|3450294021839982|
|95329602 + 45418854|140748456|
|412907 + 3731910|4144817|
|...|...|

You will have several challenges.

* You must create your prompt so that it calculates the equation, and returns JUST A NUMBER. You might also use Python to trim the result to just a number.
* You must use a tool, the LLM will not be accurate alone.

Make use of the code for python_repl that I provided in module 7.2 for tools.


# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
import os

try:
  from google.colab import drive, userdata
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google CoLab")
except:
  print("Note: not using Google CoLab")
  COLAB = False

# Assignment Submission Key - Was sent you first week of class.
# If you are in both classes, this is the same key.
if COLAB:
  # For Colab, add to your "Secrets" (key icon at the left)
  key = userdata.get('T81_559_KEY')
else:
  # If not colab, enter your key here, or use an environment variable.
  # (this is only an example key, use yours)
  key = ""

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io
from typing import List, Union

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 10.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.

def submit(
    data: List[Union[pd.DataFrame, PIL.Image.Image]],
    key: str,
    course: str,
    no: int,
    source_file: str = None
) -> None:
    if source_file is None and '__file__' not in globals():
        raise Exception("Must specify a filename when in a Jupyter notebook.")
    if source_file is None:
        source_file = __file__

    suffix = f'_class{no}'
    if suffix not in source_file:
        raise Exception(f"{suffix} must be part of the filename.")

    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception(f"Source file is {ext}; must be .py or .ipynb")

    with open(source_file, "rb") as file:
        encoded_python = base64.b64encode(file.read()).decode('ascii')

    payload = []
    for item in data:
        if isinstance(item, PIL.Image.Image):
            buffered = io.BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG': base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif isinstance(item, pd.DataFrame):
            payload.append({'CSV': base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
        else:
            raise ValueError(f"Unsupported data type: {type(item)}")

    response = requests.post(
        "https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key': key},
        json={
            'payload': payload,
            'assignment': no,
            'course': course,
            'ext': ext,
            'py': encoded_python
        }
    )

    if response.status_code == 200:
        print(f"Success: {response.text}")
    else:
        print(f"Failure: {response.text}")

# Assignment #7 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
import os
import pandas as pd
from scipy.stats import zscore
import string
from langchain.prompts import ChatPromptTemplate
from decimal import Decimal, getcontext
from langchain.tools import StructuredTool
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import Tool
import re
from langchain_experimental.utilities import PythonREPL


# You must identify your source file.  (modify for your local setup)
# file="/content/drive/My Drive/Colab Notebooks/assignment_yourname_t81_559_class7.ipynb"  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_559_deep_learning\\assignments\\assignment_yourname_t81_559_class7.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_559_deep_learning/assignments/assignment_yourname_t81_559_class7.ipynb'  # Mac/Linux

file="/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_t81_559_class7.ipynb"

# Begin assignment

df = pd.read_csv("https://data.heatonresearch.com/data/t81-559/assignments/559_numbers_2.csv")
df.head()

## ... continue your code...

#1
#
# class MathExpressionInput(BaseModel):
#     input: str

#
# def safe_calculator(input: str, **kwargs):
#     try:
#         result = eval(input, {"__builtins__": None}, {"Decimal": Decimal})
#         return str(result)
#     except Exception as e:
#         return str(e)

# # Define the tool
# safe_math_tool = StructuredTool(
#     name="safe_calc",
#     description="A math calculator used to evaluate mathematical expressions. Input should be a valid math expression.",
#     func=safe_calculator,
#     args_schema=MathExpressionInput
# )

# MODEL = 'gpt-4o-mini'
# llm = ChatOpenAI(model=MODEL, temperature=0.2, n=1)

# prompt = hub.pull("hwchase17/openai-functions-agent")

# # agent and executor
# tools = [safe_math_tool]
# agent = create_tool_calling_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# # Process equations
# def extract_numeric_result(equation):
#     output = agent_executor.invoke({"input": f"Calculate {equation} and return only the numeric result."})["output"]
#     match = re.search(r'-?\d+\.?\d*(?:[eE][-+]?\d+)?', output)
#     return match.group() if match else "ERROR"

# df["value"] = df["equation"].apply(extract_numeric_result)

# # Convert 'value' column to numeric type
# df["value"] = pd.to_numeric(df["value"], errors='coerce')
#1

## ... continue your code...

# ##

python_repl = PythonREPL()

# Define the tool using Python REPL
def run_python_code(input: str):
    return python_repl.run(input)

repl_tool = Tool(
    name="python_repl",
    description="A Python shell for executing commands. Input should be a valid Python expression.",
    func=run_python_code
)

MODEL = 'gpt-4o-mini'
llm = ChatOpenAI(model=MODEL, temperature=0.2, n=1)

# Load prompt
prompt = hub.pull("hwchase17/openai-functions-agent")


tools = [repl_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Process equations
def extract_numeric_result(equation):
    output = agent_executor.invoke({"input": f"Calculate {equation} and return only the numeric result."})["output"]
    match = re.search(r'-?\d+\.?\d*(?:[eE][-+]?\d+)?', output)
    return match.group() if match else "ERROR"

df["value"] = df["equation"].apply(extract_numeric_result)

df["value"] = pd.to_numeric(df["value"], errors='coerce')


df_submit = df

# df_submit
## Submit assignment

# Submit
submit(source_file=file,data=[df_submit],course='t81-559',key=key,no=7)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`



Invoking: `python_repl` with `41748459 - 87226336`




> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl` with `92995162 * 46769739`



Invoking: `python_repl` with `92995